In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# EXERCISE 1 SOLUTION
num_qubits = 2

amplitudes = cudaq.get_state(nested_quantum_program, num_qubits).amplitudes([[1,0], [1,1]])
print('|Psi>= {} |10> + {} |11>'.format(np.round(amplitudes[0],precision), np.round(amplitudes[1],precision)))

In [ ]:
# EXERCISE 2 SOLUTION

num_qubits = 3
shots = 1000

@cudaq.kernel
def alternating_signs(qubit_count: int):
    qubits = cudaq.qvector(num_qubits)
    for i in range(num_qubits):
        if i % 2 != 0:
            x(qubits[i])
    h(qubits)
        
# Draw the circuit
print(cudaq.draw(alternating_signs,num_qubits))

# Verify state
# Compute the state of the system prior to measurement
state = cudaq.get_state(alternating_signs, num_qubits)

# Print 
precision = 4
print('Statevector array of coefficients:', np.round(np.array(state), precision))

In [ ]:

# EXERCISE 3 Part a SOLUTION
@cudaq.kernel
def alternating_cnots(qubit0: cudaq.qubit, qubit1: cudaq.qubit):
    """Apply a sequence of 3 CNOTs with alternating controls and targets on the 2 qubits given as arguments"""
    # We will see later that it doesn't matter which qubit you start out with as the first control, as long as
    # you alternate the control and target qubits with each of the 3 applications of the cnot gate
    
    # Edit code below this line
    x.ctrl(qubit0, qubit1)
    x.ctrl(qubit1, qubit0)
    x.ctrl(qubit0, qubit1)
    # Edit code above this line


@cudaq.kernel
def three_alternating_cnots():
    """Kernel for the circuit drawn above"""
    
    # Allocate qubits
    q = cudaq.qvector(2)
    
    # Initialize qubits q0 and q1 in the plus and one states, respectively
    h(q[0])
    x(q[1])
    
    # Apply alternating CNOTs
    alternating_cnots(q[0], q[1])
    

    
results = cudaq.sample(three_alternating_cnots)
print('The distribution of states after sampling is: {}'.format(results))


In [ ]:
# EXERCISE 3 Part b SOLUTION

@cudaq.kernel
def apply_cnot(control: cudaq.qubit, target: cudaq.qubit):
    """Apply an CNOT gate to the control and target qubits"""
    x.ctrl(control, target)

@cudaq.kernel
def initialize_plus_zero(qubits: cudaq.qview):
    """Kernel to initialize the state indicated by the given list of bits"""
    # Place qubits[0] in the plus state
    x(qubits[0])
    h(qubits[0])

@cudaq.kernel
def CNOT_exercise():
    """Apply CNOT to |+0> with control q0 and target q1"""
    qubits =cudaq.qvector(2)
    # Initialize state
    initialize_plus_zero(qubits)
    # Apply CNOT to the first two bits
    apply_cnot(qubits[0], qubits[1])

results = cudaq.sample(CNOT_exercise)
print('CNOT applied to the state |+0> results in the distribution of states: {}'.format(results))


In [ ]:
# EXERCISE 4 SOLUTION
num_qubits = 3

@cudaq.kernel
def initial_state(qubits : cudaq.qview):
    for index in range(len(qubits)):
        if index % 2 !=0:
            x(qubits[index])
    h(qubits)   

@cudaq.kernel
def interference(qubit_count: int):
    qvector = cudaq.qvector(qubit_count)
    
    initial_state(qvector) # Initialize the state
    
    # Apply x.ctrl with control q_0 and target q_1. Then apply a Hadamard gate to q_1.
    # Edit the code below this line
    x.ctrl(qvector[0], qvector[1])
    h(qvector[1])
    # Edit the code above this line 
    
results = cudaq.sample(interference, num_qubits, shots_count = 1000)
print(results)

In [ ]:
# EXERCISE 5 SOLUTION

@cudaq.kernel
def initialize_state(qubit : cudaq.qubit):
    # Edit the code below to initialize the qubit in a state that you can use to distinguish
    # the t and s gates. 
    # Hint: t and s gates both perform a rotation about the z axis.
    h(qubit) # Any initial state on the xy-plane of the Bloch Sphere would work
    # Edit the code above

    
@cudaq.kernel
def compare_t_s(t_or_s : int):
    qubit = cudaq.qubit()
    
    initialize_state(qubit)
    
    if t_or_s == 0:
        t(qubit)
    else:
        s(qubit)
    
# Define a sphere object representing the state of the single initialized qubit after an application of t or s
t_or_s = 1 #Set the variable = 0 to apply t; set it equal to 1 to apply
assert (t_or_s == 0 or t_or_s == 1)
sphere = cudaq.add_to_bloch_sphere(cudaq.get_state(compare_t_s, t_or_s))

# Display the Bloch sphere
cudaq.show(sphere)


In [ ]:
# EXERCISE 6 SOLUTION
import random

@cudaq.kernel
def modular_mult_5_21(qubits : cudaq.qvector):
    """"Kernel based off of the circuit diagram in
    https://physlab.org/wp-content/uploads/2023/05/Shor_s_Algorithm_23100113_Fin.pdf
    Modifications were made to change the ordering of the qubits.
    """

    x(qubits[0])
    x(qubits[2])
    x(qubits[4])

    swap(qubits[0], qubits[4])
    swap(qubits[0], qubits[2])

@cudaq.kernel
def encode_integer(qubits: cudaq.qvector, binary_rep: list[int]):
    """Kernel takes as input a list of qubits and the binary representation 
    of an integer as a list. The kernel adds X-gates to the qubits to encode
    the binary list, placing an X gate on qubit i if there is a 1 in the ith location
    on the binary_rep list"""
    # Edit code below this line
    for i in range(len(binary_rep)):
        if binary_rep[i] == 1:
            x(qubits[i])
    # Edit code above this line


def decimal_to_binary_list(number):
    # Check if the input number is valid (non-negative integer)
    if number < 0:
        raise ValueError("Number must be a non-negative integer.")

    # Convert the number to binary using bin() function and strip the '0b' prefix
    binary_string = bin(number)[2:]

    # Convert the binary string to a list of integers (0s and 1s)
    binary_list = [int(bit) for bit in binary_string]

    return binary_list

@cudaq.kernel
def mult_y_by_5_mod21(binary_list: list[int]):
    
    # Allocate qubits
    qubits = cudaq.qvector(5)
    
    # Initialize qubits in the state representing 1, 4, 5, 16, 17, or 20
    encode_integer(qubits, binary_list)
    # Apply the multiplication by 5 mod 21 kernel
    modular_mult_5_21(qubits)

values = [1,4,5,16, 17, 20]
number = random.choice(values) # selects a number from the list values randomly
# Convert number into a binary representation stored as a list of 0s and 1s
binary_list_reversed = decimal_to_binary_list(number)
binary_list = binary_list_reversed[::-1]  # Reverse the list to match qubit order

results = cudaq.sample(mult_y_by_5_mod21, binary_list, shots_count = 200)
print("Multiplying {} by 5 mod 21 results in the bitstring {}".format(number,results.most_probable()))

In [ ]:
# EXERCISE 7 SOLUTION

@cudaq.kernel
def modular_exp_kernel(exponent: int):
    """Kernel computes 5^x mod 21
    Parameters:
    -----------
    exponent : int
        the value x for the computation 5^x mod 21
    
    Returns:
    --------
    binary_rep : string
        binary representation for 5^x mod 21 as a string of 0s and 1s    
    """
    # Allocate and intialize qubits
    qubits = cudaq.qvector(5)
    
    # Edit code below this line
    
    # Encode y = 1 
    x(qubits[0])
    
    # Multiply y = 1 by 5 exp times
    
    for n in range(0,exponent):
        modular_mult_5_21(qubits)    
    
    # Edit code above this line


def modular_exp(exponent: int):
    sample_result = cudaq.sample(modular_exp_kernel, exponent, shots_count = 100).most_probable()
    return sample_result

for x in range(0,7):
    result = modular_exp(x)
    
    print("5^{} mod 21 in binary representation is {}".format(x, result))